Install environment management tools for API keys

In [5]:
!pip install python-dotenv
!pip install colab-env -qU

import colab_env
from colab_env import envvar_handler 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import requests
import os
import pandas as pd
from dotenv import load_dotenv

# ALCHEMY_API_KEY_TREASURE

import requests
load_dotenv()

ALCHEMY_API_KEY_TREASURE = os.environ['ALCHEMY_API_KEY_TREASURE']


url = f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY_TREASURE}"


payload = {
    "id": 1,
    "jsonrpc": "2.0",
    "method": "alchemy_getAssetTransfers",
    "params": [
        {
            "fromBlock": "0x0",
            "toBlock": "latest",
            "category": ["erc20"],
            "withMetadata": False,
            "excludeZeroValue": True,
            "maxCount": "0x3e8",
            "fromAddress": "0xa8b0a855BE21568B93f32805b244158Cc61AD006"
        }
    ]
}
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

# print(response.text)

Define Functions
---
Define the functions for API calls. 

get_transfers_wo_pgkey: this is called first and does not take a 'page_key' input.

get_transfers_w_pgkey: can be called once a 'page_key' has been obtained

In [9]:
import requests
import os
import pandas as pd
from dotenv import load_dotenv

# drive.mount('/content/drive/') 

load_dotenv()
ALCHEMY_API_KEY_TREASURE = os.environ['ALCHEMY_API_KEY_TREASURE']


ALCHEMY_API_KEY_TREASURE = os.getenv('ALCHEMY_API_KEY_TREASURE')
url = f"https://arb-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY_TREASURE}"

# print(ALCHEMY_API_KEY_TREASURE)

def get_transfers_w_pgkey(_PAGE_KEY):
  PAGE_KEY = _PAGE_KEY
  payload = {
        "id": 1,
        "jsonrpc": "2.0",
        "method": "alchemy_getAssetTransfers",
        "params": [
            {
                "fromBlock": "0x0",
                "toBlock": "latest",
                "contractAddresses": ["0xfE8c1ac365bA6780AEc5a985D989b327C27670A1"],
                "category": ["erc721"],
                "withMetadata": True,
                "excludeZeroValue": False,
                "maxCount": "0x3e8",
                "pageKey": f"{PAGE_KEY}"
            }
        ]
    }
  headers = {
      "Accept": "application/json",
      "Content-Type": "application/json"
    }

  response = requests.post(url, json=payload, headers=headers)
  return response

def get_transfers_wo_pgkey():
  payload = {
      "id": 1,
      "jsonrpc": "2.0",
      "method": "alchemy_getAssetTransfers",
      "params": [
          {
              "fromBlock": "0x0",
              "toBlock": "latest",
              "contractAddresses": ["0xfE8c1ac365bA6780AEc5a985D989b327C27670A1"],
              "category": ["erc721"],
              "withMetadata": True,
              "excludeZeroValue": False,
              "maxCount": "0x3e8"
          }
      ]
  }
  headers = {
      "Accept": "application/json",
      "Content-Type": "application/json"
  }

  response = requests.post(url, json=payload, headers=headers)
  return response


# print(response.text)

Get All Transfer Data
---
Use a loop that checks for a page_key to capture all transfer data. If there is no additional data pg_key will be empty and will return false.

In [11]:
import time
response=get_transfers_wo_pgkey()
page_key = response.json()['result']['pageKey']
result = response.json()['result']['transfers']
concat_num =0
df_new = pd.DataFrame()
df_transfers = pd.json_normalize(result)
df_new = df_transfers
# while df_new.shape[0] == 1000:
while page_key:
    response=get_transfers_w_pgkey(page_key)
    if 'error' in response.json(): 
        try:
            response=get_transfers_w_pgkey(page_key)
        except:
            print(response.json()['error']['code'])
    try:
        page_key = response.json()['result']['pageKey']
    # response=get_transfers_w_pgkey(page_key)
        result = response.json()['result']['transfers']
    except:
        page_key = False
        # print(response.text)
        print("page key not found")
  

    df_new = pd.json_normalize(result)

    df_transfers=pd.concat([df_transfers, df_new], axis=0)
    concat_num = concat_num + 1
    if concat_num % 25 == 0:
        print(f"Iteration Number: {concat_num}")
    if concat_num %200 == 0:
        time.sleep(5)
        print('sleeping')

# df_transfers
# response = get_transfers()
# response.json()

# df = pd.json_normalize(response.json()['result']['transfers'])


Iteration Number: 25
Iteration Number: 50
Iteration Number: 75
Iteration Number: 100
Iteration Number: 125
Iteration Number: 150
Iteration Number: 175
Iteration Number: 200
sleeping
Iteration Number: 225
Iteration Number: 250
Iteration Number: 275
Iteration Number: 300
Iteration Number: 325
Iteration Number: 350
Iteration Number: 375
Iteration Number: 400
sleeping
Iteration Number: 425
Iteration Number: 450
Iteration Number: 475
Iteration Number: 500
Iteration Number: 525
Iteration Number: 550
Iteration Number: 575
Iteration Number: 600
sleeping
Iteration Number: 625
Iteration Number: 650
Iteration Number: 675
Iteration Number: 700
Iteration Number: 725
Iteration Number: 750
Iteration Number: 775
Iteration Number: 800
sleeping
Iteration Number: 825
Iteration Number: 850
Iteration Number: 875
page key not found


In [12]:
df_transfers

,blockNum,hash,from,to,value,erc721TokenId,erc1155Metadata,tokenId,asset,category,rawContract.value,rawContract.address,rawContract.decimal,metadata.blockTimestamp
0,0x4c9656,0xfa05eebfcc91297193fdba3a06b6dd5cc0275c54f4c2...,0x0000000000000000000000000000000000000000,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,None,0x00000000000000000000000000000000000000000000...,None,0x00000000000000000000000000000000000000000000...,LGN,erc721,None,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,None,2022-01-25T12:16:23.000Z
1,0x4c98ba,0xb8a4b226015a0739ef641a23376180f621a36ec1a97e...,0x0000000000000000000000000000000000000000,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,None,0x00000000000000000000000000000000000000000000...,None,0x00000000000000000000000000000000000000000000...,LGN,erc721,None,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,None,2022-01-25T12:35:45.000Z
2,0x4c9b04,0x32f8dfb9b13b30c24a254852570b9be8e2625b2ce7e9...,0x0000000000000000000000000000000000000000,0x032f84aeff59ddebc55797f321624826d873bf65,None,0x00000000000000000000000000000000000000000000...,None,0x00000000000000000000000000000000000000000000...,LGN,erc721,None,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,None,2022-01-25T12:52:12.000Z
3,0x4cefa9,0x2b222da87c15b534bddccbb5731a7e34d642c7ec2d58...,0x0000000000000000000000000000000000000000,0xa82fcefd303fa68864b787a5f118b09ce5a4c93d,None,0x00000000000000000000000000000000000000000000...,None,0x00000000000000000000000000000000000000000000...,LGN,erc721,None,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,None,2022-01-25T21:51:31.000Z
4,0x4cf6fb,0x393d45524a689884d342a333f2fac06faf57aaed1a54...,0x0000000000000000000000000000000000000000,0x711aca028ecaea178ebc29c7059cfdb195facd37,None,0x00000000000000000000000000000000000000000000...,None,0x00000000000000000000000000000000000000000000...,LGN,erc721,None,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,None,2022-01-25T23:06:55.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0x140fae3,0xd06ac66cabc7a235b3bcbe1c3e941a12bce136891a5f...,0xa0a89db1c899c49f98e6326b764bafcf167fc2ce,0x6c7de01cc27104709f6204ceb547ec0d33beccae,None,0x00000000000000000000000000000000000000000000...,None,0x00000000000000000000000000000000000000000000...,LGN,erc721,None,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,None,2022-08-23T09:04:15.000Z
996,0x140fb0f,0x1597379e778e902cfbeb628433e09dfa1cad4c66920b...,0xb9c9ed651eb173ca7fbc3a094da9ce33ec145a29,0xf1386d9e197b8461c6a4b12e70d7106c27d40f3c,None,0x00000000000000000000000000000000000000000000...,None,0x00000000000000000000000000000000000000000000...,LGN,erc721,None,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,None,2022-08-23T09:04:49.000Z
997,0x140fb42,0x3e35b74fafa79a16af1c360cd281fc0c1c965d6f6dbc...,0x19a015963dfd61173ef2fbcff36877db4a19160b,0x2db620dda33c246db599575d580a1559cee1e97d,None,0x00000000000000000000000000000000000000000000...,None,0x00000000000000000000000000000000000000000000...,LGN,erc721,None,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,None,2022-08-23T09:05:43.000Z
998,0x140fb6a,0x74b7b08e0fa036b8cbeca3043bc8b1c8eb7deb939277...,0xf1386d9e197b8461c6a4b12e70d7106c27d40f3c,0xb9c9ed651eb173ca7fbc3a094da9ce33ec145a29,None,0x00000000000000000000000000000000000000000000...,None,0x00000000000000000000000000000000000000000000...,LGN,erc721,None,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,None,2022-08-23T09:05:43.000Z


In [13]:
# df_transfers = pd.json_normalize(result)
df_transfers['blockNum'] = df_transfers['blockNum'].apply(int, base=16)
df_transfers['erc721TokenId']=df_transfers['erc721TokenId'].apply(int, base=16)
df_transfers = df_transfers[['blockNum', 'hash', 'from', 'to', 'erc721TokenId', 'tokenId', 'asset', 'rawContract.address', 'metadata.blockTimestamp']]
df_transfers

,blockNum,hash,from,to,erc721TokenId,tokenId,asset,rawContract.address,metadata.blockTimestamp
0,5019222,0xfa05eebfcc91297193fdba3a06b6dd5cc0275c54f4c2...,0x0000000000000000000000000000000000000000,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,1,0x00000000000000000000000000000000000000000000...,LGN,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,2022-01-25T12:16:23.000Z
1,5019834,0xb8a4b226015a0739ef641a23376180f621a36ec1a97e...,0x0000000000000000000000000000000000000000,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,2,0x00000000000000000000000000000000000000000000...,LGN,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,2022-01-25T12:35:45.000Z
2,5020420,0x32f8dfb9b13b30c24a254852570b9be8e2625b2ce7e9...,0x0000000000000000000000000000000000000000,0x032f84aeff59ddebc55797f321624826d873bf65,3,0x00000000000000000000000000000000000000000000...,LGN,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,2022-01-25T12:52:12.000Z
3,5042089,0x2b222da87c15b534bddccbb5731a7e34d642c7ec2d58...,0x0000000000000000000000000000000000000000,0xa82fcefd303fa68864b787a5f118b09ce5a4c93d,4,0x00000000000000000000000000000000000000000000...,LGN,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,2022-01-25T21:51:31.000Z
4,5043963,0x393d45524a689884d342a333f2fac06faf57aaed1a54...,0x0000000000000000000000000000000000000000,0x711aca028ecaea178ebc29c7059cfdb195facd37,5,0x00000000000000000000000000000000000000000000...,LGN,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,2022-01-25T23:06:55.000Z
...,...,...,...,...,...,...,...,...,...
995,21035747,0xd06ac66cabc7a235b3bcbe1c3e941a12bce136891a5f...,0xa0a89db1c899c49f98e6326b764bafcf167fc2ce,0x6c7de01cc27104709f6204ceb547ec0d33beccae,9994,0x00000000000000000000000000000000000000000000...,LGN,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,2022-08-23T09:04:15.000Z
996,21035791,0x1597379e778e902cfbeb628433e09dfa1cad4c66920b...,0xb9c9ed651eb173ca7fbc3a094da9ce33ec145a29,0xf1386d9e197b8461c6a4b12e70d7106c27d40f3c,31987,0x00000000000000000000000000000000000000000000...,LGN,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,2022-08-23T09:04:49.000Z
997,21035842,0x3e35b74fafa79a16af1c360cd281fc0c1c965d6f6dbc...,0x19a015963dfd61173ef2fbcff36877db4a19160b,0x2db620dda33c246db599575d580a1559cee1e97d,20998,0x00000000000000000000000000000000000000000000...,LGN,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,2022-08-23T09:05:43.000Z
998,21035882,0x74b7b08e0fa036b8cbeca3043bc8b1c8eb7deb939277...,0xf1386d9e197b8461c6a4b12e70d7106c27d40f3c,0xb9c9ed651eb173ca7fbc3a094da9ce33ec145a29,31987,0x00000000000000000000000000000000000000000000...,LGN,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,2022-08-23T09:05:43.000Z


In [14]:
df_transfers = df_transfers[['blockNum', 'hash', 'from', 'to', 'erc721TokenId', 'asset', 'rawContract.address', 'metadata.blockTimestamp']]

df_transfers

,blockNum,hash,from,to,erc721TokenId,asset,rawContract.address,metadata.blockTimestamp
0,5019222,0xfa05eebfcc91297193fdba3a06b6dd5cc0275c54f4c2...,0x0000000000000000000000000000000000000000,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,1,LGN,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,2022-01-25T12:16:23.000Z
1,5019834,0xb8a4b226015a0739ef641a23376180f621a36ec1a97e...,0x0000000000000000000000000000000000000000,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,2,LGN,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,2022-01-25T12:35:45.000Z
2,5020420,0x32f8dfb9b13b30c24a254852570b9be8e2625b2ce7e9...,0x0000000000000000000000000000000000000000,0x032f84aeff59ddebc55797f321624826d873bf65,3,LGN,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,2022-01-25T12:52:12.000Z
3,5042089,0x2b222da87c15b534bddccbb5731a7e34d642c7ec2d58...,0x0000000000000000000000000000000000000000,0xa82fcefd303fa68864b787a5f118b09ce5a4c93d,4,LGN,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,2022-01-25T21:51:31.000Z
4,5043963,0x393d45524a689884d342a333f2fac06faf57aaed1a54...,0x0000000000000000000000000000000000000000,0x711aca028ecaea178ebc29c7059cfdb195facd37,5,LGN,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,2022-01-25T23:06:55.000Z
...,...,...,...,...,...,...,...,...
995,21035747,0xd06ac66cabc7a235b3bcbe1c3e941a12bce136891a5f...,0xa0a89db1c899c49f98e6326b764bafcf167fc2ce,0x6c7de01cc27104709f6204ceb547ec0d33beccae,9994,LGN,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,2022-08-23T09:04:15.000Z
996,21035791,0x1597379e778e902cfbeb628433e09dfa1cad4c66920b...,0xb9c9ed651eb173ca7fbc3a094da9ce33ec145a29,0xf1386d9e197b8461c6a4b12e70d7106c27d40f3c,31987,LGN,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,2022-08-23T09:04:49.000Z
997,21035842,0x3e35b74fafa79a16af1c360cd281fc0c1c965d6f6dbc...,0x19a015963dfd61173ef2fbcff36877db4a19160b,0x2db620dda33c246db599575d580a1559cee1e97d,20998,LGN,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,2022-08-23T09:05:43.000Z
998,21035882,0x74b7b08e0fa036b8cbeca3043bc8b1c8eb7deb939277...,0xf1386d9e197b8461c6a4b12e70d7106c27d40f3c,0xb9c9ed651eb173ca7fbc3a094da9ce33ec145a29,31987,LGN,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,2022-08-23T09:05:43.000Z


In [15]:
df_transfer_to = df_transfers[['metadata.blockTimestamp', 'blockNum', 'hash', 'to', 'erc721TokenId','asset', 'rawContract.address']]
df_transfer_to['value'] = 1
df_transfer_to=df_transfer_to.rename(columns = {'metadata.blockTimestamp':'timestamp', 'blockNum':'block_num', 'to':'address', 'rawContract.address':'contract_address'})
df_transfer_from = df_transfers[['metadata.blockTimestamp', 'blockNum', 'hash', 'from', 'erc721TokenId','asset', 'rawContract.address']]
df_transfer_from['value'] = -1
df_transfer_from=df_transfer_from.rename(columns = {'metadata.blockTimestamp':'timestamp', 'blockNum':'block_num', 'from':'address', 'rawContract.address':'contract_address'})



In [16]:
# df_transfer_to.merge(df_transfer_from, how='left', left_on='to', right_on='from')
pd_transfer_w_value = pd.concat([df_transfer_to, df_transfer_from], axis=0)

In [17]:
addresses = pd.DataFrame(pd_transfer_w_value['address'].unique())
days = pd.to_datetime(pd_transfer_w_value['timestamp']).dt.date.unique()
days = pd.date_range(start=days.min(), end=days.max(), freq="D")


In [18]:
df_test = pd.DataFrame(
    {
    'day': days
    # 'addresses': addresses
    }
)
addresses['key'] = 1
df_test['key']=1
result=df_test.merge(addresses, on='key').drop(columns=['key'])

In [19]:
r=result.reset_index().rename(columns={'day':'date', 0:'address'})
r['date'] = pd.to_datetime(r['date']).dt.date
r

,index,date,address
0,0,2022-01-25,0x16e95f6bf27f0b16b19ad7d07635e69c49897272
1,1,2022-01-25,0x032f84aeff59ddebc55797f321624826d873bf65
2,2,2022-01-25,0xa82fcefd303fa68864b787a5f118b09ce5a4c93d
3,3,2022-01-25,0x711aca028ecaea178ebc29c7059cfdb195facd37
4,4,2022-01-25,0x259f962e77262ee8d97cc6a112d0e0321323ea0d
...,...,...,...
3586573,3586573,2022-08-23,0xf1dbbd6864fcb13e24d68a80111609cf0ffa08c0
3586574,3586574,2022-08-23,0xf037e10e4d6067b1f75a9b8b5fdd4ce78fe3b16d
3586575,3586575,2022-08-23,0x468e60341ee7ab367244f6a9bb7e1a3511fd11a2
3586576,3586576,2022-08-23,0xc48cd44e45c9324801a2c067dc7b48723f648d3b


In [20]:
# grp=pd_transfer_w_value.groupby(['timestamp', 'address']).sum()
grp=pd_transfer_w_value.reset_index()
grp['day'] = pd.to_datetime(grp['timestamp']).dt.date
grp.sort_values('timestamp', inplace=True)
grp.set_index('timestamp', inplace=True)
grp=grp.drop(columns=['index', 'block_num', 'hash', 'asset', 'erc721TokenId'])


# grp['daily_change']=grp.groupby('day').sum()
daily_change = grp.groupby(['day', 'address']).sum().reset_index()

In [27]:
grp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1796000 entries, 2022-01-25T12:16:23.000Z to 2022-08-23T09:05:43.000Z
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   address           object
 1   contract_address  object
 2   value             int64 
 3   day               object
dtypes: int64(1), object(3)
memory usage: 68.5+ MB


In [21]:
daily_change.reset_index().head(50)

,index,day,address,value
0,0,2022-01-25,0x0000000000000000000000000000000000000000,-142
1,1,2022-01-25,0x01c38875ab789cb12efcd791db91217a00fd01fa,1
2,2,2022-01-25,0x032f84aeff59ddebc55797f321624826d873bf65,1
3,3,2022-01-25,0x0578aa0595e0c83530dcde1d7a339479bfe6b0b7,1
4,4,2022-01-25,0x064e2e9c7df51861ac4492551baf1fd6e3286074,1
5,5,2022-01-25,0x12c6082bedfc53f2afb059cd85b9d63dcf0ac5a9,1
6,6,2022-01-25,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,2
7,7,2022-01-25,0x259f962e77262ee8d97cc6a112d0e0321323ea0d,13
8,8,2022-01-25,0x29059f184faa0a32c83971c8951829c8be27d3f3,4
9,9,2022-01-25,0x2f1fbfd195854f6d0bc45d1e420c2d46549eb624,3


In [22]:
daily_change_all_days = r.merge(daily_change, how='left', left_on=['date','address'], right_on=['day','address']).fillna(0)
daily_change_all_days=daily_change_all_days[['date', 'address', 'value']]

Daily Wallet Balances & Net Movement
---
Get the net change in tokens and wallet balance for each address

In [39]:
daily_change_all_days['cumsum'] = daily_change_all_days.groupby(['address'])['value'].cumsum()
daily_change_all_days.to_csv('legion_holders_by_day.csv')
daily_change_all_days

,date,address,value,cumsum
0,2022-01-25,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,2.0,2.0
1,2022-01-25,0x032f84aeff59ddebc55797f321624826d873bf65,1.0,1.0
2,2022-01-25,0xa82fcefd303fa68864b787a5f118b09ce5a4c93d,2.0,2.0
3,2022-01-25,0x711aca028ecaea178ebc29c7059cfdb195facd37,1.0,1.0
4,2022-01-25,0x259f962e77262ee8d97cc6a112d0e0321323ea0d,13.0,13.0
...,...,...,...,...
3586573,2022-08-23,0xf1dbbd6864fcb13e24d68a80111609cf0ffa08c0,0.0,0.0
3586574,2022-08-23,0xf037e10e4d6067b1f75a9b8b5fdd4ce78fe3b16d,0.0,0.0
3586575,2022-08-23,0x468e60341ee7ab367244f6a9bb7e1a3511fd11a2,0.0,80.0
3586576,2022-08-23,0xc48cd44e45c9324801a2c067dc7b48723f648d3b,0.0,0.0


In [24]:
daily_change_all_days[daily_change_all_days['address'] == '0x16e95f6bf27f0b16b19ad7d07635e69c49897272'].head(30)

,date,address,value,cumsum
0,2022-01-25,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,2.0,2.0
16998,2022-01-26,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,1.0,3.0
33996,2022-01-27,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,0.0,3.0
50994,2022-01-28,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,0.0,3.0
67992,2022-01-29,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,0.0,3.0
84990,2022-01-30,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,0.0,3.0
101988,2022-01-31,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,0.0,3.0
118986,2022-02-01,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,-3.0,0.0
135984,2022-02-02,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,0.0,0.0
152982,2022-02-03,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,1.0,1.0


In [38]:
# wallet_bal_by_day = grp.drop(columns=['timestamp', 'block_num', 'hash', 'erc721TokenId', 'asset'])
wallet_bal_by_day = grp

# wallet_bal_by_day=wallet_bal_by_day[['day', 'contract_address','address', 'value', 'cumsum']]
wallet_bal_by_day

,address,contract_address,value,day
timestamp,,,,
2022-01-25T12:16:23.000Z,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,1,2022-01-25
2022-01-25T12:16:23.000Z,0x0000000000000000000000000000000000000000,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,-1,2022-01-25
2022-01-25T12:35:45.000Z,0x16e95f6bf27f0b16b19ad7d07635e69c49897272,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,1,2022-01-25
2022-01-25T12:35:45.000Z,0x0000000000000000000000000000000000000000,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,-1,2022-01-25
2022-01-25T12:52:12.000Z,0x032f84aeff59ddebc55797f321624826d873bf65,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,1,2022-01-25
...,...,...,...,...
2022-08-23T09:05:43.000Z,0x2a7051d7cbbef7b6889f8e14774020b1653b94c1,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,1,2022-08-23
2022-08-23T09:05:43.000Z,0xb9c9ed651eb173ca7fbc3a094da9ce33ec145a29,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,1,2022-08-23
2022-08-23T09:05:43.000Z,0x2db620dda33c246db599575d580a1559cee1e97d,0xfe8c1ac365ba6780aec5a985d989b327c27670a1,1,2022-08-23


In [ ]:
daily_change_all_days.tail(100)

Unique Addresses with LGN Tokens
---
Get the number of addresses holding LGN tokens each day

In [35]:
unique_holders = daily_change_all_days[(daily_change_all_days['cumsum'] > 0)].groupby('date').size()
unique_holders=unique_holders.reset_index()
unique_holders.rename(columns={0:'unique_holders'}, inplace=True)

In [36]:
unique_holders

,date,unique_holders
0,2022-01-25,65
1,2022-01-26,772
2,2022-01-27,769
3,2022-01-28,762
4,2022-01-29,771
...,...,...
206,2022-08-19,3768
207,2022-08-20,3766
208,2022-08-21,3764
209,2022-08-22,3772


In [37]:
unique_holders.to_csv('unique_legion_holders.csv')

In [32]:
unique_holders

<class 'pandas.core.series.Series'>
Index: 211 entries, 2022-01-25 to 2022-08-23
Series name: None
Non-Null Count  Dtype
--------------  -----
211 non-null    int64
dtypes: int64(1)
memory usage: 3.3+ KB
